In [1]:
import scarches as sca
import pickle as pkl
import scvi
import scanpy as sc
import os
import numpy as np
from dotenv import load_dotenv
from lightning.pytorch.loggers import WandbLogger

from pyprojroot import here

overwriteData = True

/home/dmaspero/miniconda3/envs/scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


In [2]:
assert load_dotenv()

### Loading data

In [3]:
reference_adata = sc.read_h5ad(here(f"09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_model_fineTuned_lowLR_batches/adata.h5ad"))

external_adata = sc.read_h5ad(here("R3/results/SCGT00_EXTERNAL_afterQC.h5ad"))

**Keep only selected genes**

In [4]:
external_adata = external_adata[:,reference_adata.var.index].copy()

In [5]:
reference_adata, external_adata

(AnnData object with n_obs × n_vars = 756120 × 8253
     obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'batches', 'Level1', '_scvi_batch', '_scvi_labels'
     var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status'
     uns: '_scvi_manager_uuid', '_scvi_uuid',
 AnnData object with n_obs × n_vars = 379359 × 8253
     obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'batches', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt'


In [6]:
scanvi_model = scvi.model.SCANVI.load(here(f"09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_model_fineTuned_lowLR_batches/"), 
                                      adata=reference_adata)

INFO     File                                                                                                      
         /scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/04_visualizing_final_embedding_space/SCGT00_CentralizedDataset/02_scANVI_integration_wit
         h_annotation/results/scANVI_model_fineTuned_lowLR_batches/model.pt already downloaded                     


Outdated cuSPARSE installation found.
Version JAX was built against: 12200
Minimum supported: 12100
Installed version: 12002
The local installation version must be no lower than 12100. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


### Parametrize model

In [7]:
model = scvi.model.SCANVI.load_query_data(
    external_adata,
    scanvi_model,
    freeze_dropout = True,
)
model._unlabeled_indices = np.arange(external_adata.n_obs)
model._labeled_indices = []

/home/dmaspero/miniconda3/envs/scarches/lib/python3.9/site-packages/scvi/data/_manager.py:209: UserWarning: Missing labels key Level1. Filling in with unlabeled category unknown.
  field_registry[_constants._STATE_REGISTRY_KEY] = field.transfer_field(


In [8]:
parameter_dict = {}
trainer_kwargs = dict(
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 10,
    early_stopping = True,
    max_epochs = 100,
    batch_size = 128 # if QUERY_ADATA_NAME != 'EXTERNAL' else 127
)

plan_kwargs = dict(weight_decay=0.0)
parameter_dict.update(trainer_kwargs)
parameter_dict.update(plan_kwargs)

In [9]:
logger = WandbLogger(
    project='inflammation_atlas_PatientClassifier_scANVI', 
    entity='inflammation',
    config=parameter_dict,
    name = f'scANVI_query_test_SCGT00_EXTERNAL',
)

### Querying external

In [10]:
model.train(
    logger=logger, 
    plan_kwargs=plan_kwargs,
    **trainer_kwargs
)

INFO     Training for 100 epochs.                                                                                  


INFO: GPU available: True (cuda), used: True


INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True


INFO: TPU available: False, using: 0 TPU cores


INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


...

Epoch 25/100:  24%|██████████▌                                 | 24/100 [27:48<1:35:36, 75.48s/it, v_num=p2ba, train_loss_step=1.63e+3, train_loss_epoch=1.88e+3]

Epoch 25/100:  25%|███████████                                 | 25/100 [29:05<1:34:59, 76.00s/it, v_num=p2ba, train_loss_step=1.63e+3, train_loss_epoch=1.88e+3]

Epoch 25/100:  25%|███████████                                 | 25/100 [29:05<1:34:59, 76.00s/it, v_num=p2ba, train_loss_step=2.03e+3, train_loss_epoch=1.88e+3]

Epoch 25/100:  25%|███████████                                 | 25/100 [29:05<1:27:15, 69.81s/it, v_num=p2ba, train_loss_step=2.03e+3, train_loss_epoch=1.88e+3]


Monitored metric reconstruction_loss_validation did not improve in the last 10 records. Best score: 1875.004. Signaling Trainer to stop.


In [11]:
if overwriteData:
    model.save(
        here(f"09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_SCGT00_EXTERNAL_batches_queried"), 
        overwrite = True,
        save_anndata = False)

### Label transfer

In [12]:
query_labels = model.predict(external_adata)

In [13]:
query_latents = model.get_latent_representation(
    external_adata
)
reference_latents = model.get_latent_representation(
    reference_adata
)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [14]:
query_ad = sc.AnnData(
    X=query_latents, 
    obs=(external_adata.obs.assign(labels=query_labels)))
query_ad.write(here(f"09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_SCGT00_EXTERNAL_batches_latent.h5ad"), compression='gzip')

In [15]:
reference_ad = sc.AnnData(
    X=reference_latents, 
    obs=reference_adata.obs)
reference_ad.write(here(f"09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_SCGT00_MAIN_batches_latent.h5ad"), compression='gzip')

### DELETE FROM HERE TO THE END 

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from pynndescent import NNDescent
from pyprojroot import here
import scanpy as sc

In [4]:
reference_ad = sc.read_h5ad(here(f"03_downstream_analysis/09_patient_classifier/SCGT00_CentralizedDataset/results_batches/scANVI_SCGT00_MAIN_batches_latent.h5ad"))

In [28]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from pynndescent import NNDescent

# Custom kNN classifier using pynndescent
class PyNNDescentKNN:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        self.index = None
        self.labels = None

    def fit(self, X, y):
        self.index = NNDescent(X, n_neighbors=self.n_neighbors, random_state=42)
        self.labels = y

    def predict(self, X):
        indices, _ = self.index.query(X, k=self.n_neighbors)
        predictions = []
        for neighbors in indices:
            neighbor_labels = self.labels[neighbors]
            predictions.append(np.bincount(neighbor_labels).argmax())
        return np.array(predictions)

# Load dataset (using Iris dataset as an example)
data = load_iris()
X, y = reference_ad[~reference_ad.obs.Level1.isin(['Progenitors','Cycling','RBC','Platelets'])].X, reference_ad[~reference_ad.obs.Level1.isin(['Progenitors','Cycling','RBC','Platelets'])].obs.disease.cat.codes

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize and train the classifier
    knn = PyNNDescentKNN(n_neighbors=30)
    knn.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = knn.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    cv_scores.append(accuracy)

# Print results
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))

/scratch_tmp/39806400/ipykernel_136436/1042024309.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/scratch_tmp/39806400/ipykernel_136436/1042024309.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = self.labels[neighbors]
/scratch_tmp/39806400/ipykernel_136436/1042024309.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[te

Cross-validation scores: [0.6323864977574947, 0.6284951966752266, 0.6292605812630991, 0.6290030686914785, 0.6321719039478108]
Mean accuracy: 0.630263449667022


In [13]:
def aggregating_features(Z = None, obsDF = None, mode = 'mean', obs_names_col = []):

    import pandas as pd
    Zdf = pd.DataFrame(Z)
    for c in obsDF.columns:
        Zdf[c] = obsDF[c].tolist()
    if mode in ['mean','avarage']:
        Zaggr = Zdf.groupby(obsDF.columns.tolist(), observed = True).mean()
    elif mode == 'sum':
        Zaggr = Zdf.groupby(obsDF.columns.tolist(), observed = True).sum()
    else:
        raise ValueError(f"mode {mode} not supported. Available mode are 'mean' or 'sum'")

    grpObs = pd.DataFrame(Zaggr.index.tolist(), columns=obsDF.columns.tolist())

    if len(obs_names_col) == 0:
        grpAdata  = sc.AnnData(X = np.array(Zaggr), obs = grpObs)
    elif all([c in obsDF.columns.tolist() for c in obs_names_col]):
        grpObs.index = grpObs[obs_names_col].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
        grpAdata  = sc.AnnData(X = np.array(Zaggr), obs = grpObs)
    else:
        raise ValueError(f"Impossible to use {obs_names_col} as index. It's not present in obsDF")
    return grpAdata

In [14]:
emb_pseudobulk_train = aggregating_features(Z = reference_ad.X, 
                                            obsDF = reference_ad.obs[['sampleID','batches','Level1','disease']], 
                                            mode = 'mean', obs_names_col=['sampleID','batches','Level1'])

In [24]:
emb_pseudobulk_train[~emb_pseudobulk_train.obs.Level1.isin(['Cycling','RBC','Platelets'])]

View of AnnData object with n_obs × n_vars = 1803 × 30
    obs: 'sampleID', 'batches', 'Level1', 'disease'

In [26]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from pynndescent import NNDescent

# Custom kNN classifier using pynndescent
class PyNNDescentKNN:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
        self.index = None
        self.labels = None

    def fit(self, X, y):
        self.index = NNDescent(X, n_neighbors=self.n_neighbors, random_state=42)
        self.labels = y

    def predict(self, X):
        indices, _ = self.index.query(X, k=self.n_neighbors)
        predictions = []
        for neighbors in indices:
            neighbor_labels = self.labels[neighbors]
            predictions.append(np.bincount(neighbor_labels).argmax())
        return np.array(predictions)

# Load dataset (using Iris dataset as an example)
data = load_iris()
X, y = reference_ad.X, reference_ad.obs.disease.cat.codes

# Set up cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize and train the classifier
    knn = PyNNDescentKNN(n_neighbors=10)
    knn.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = knn.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    cv_scores.append(accuracy)

# Print results
print("Cross-validation scores:", cv_scores)
print("Mean accuracy:", np.mean(cv_scores))

/scratch_tmp/39806400/ipykernel_136436/2794927001.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[test_index]
/scratch_tmp/39806400/ipykernel_136436/2794927001.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = self.labels[neighbors]
/scratch_tmp/39806400/ipykernel_136436/2794927001.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y[train_index], y[te

Cross-validation scores: [0.407202216066482, 0.39058171745152354, 0.45429362880886426, 0.4777777777777778, 0.4444444444444444]
Mean accuracy: 0.4348599569098184


/scratch_tmp/39806400/ipykernel_136436/2794927001.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neighbor_labels = self.labels[neighbors]
